In [1]:
import math
import struct
import inspect
from dataclasses import dataclass
from typing import Any, Optional, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

In [2]:
@dataclass
class ModelArgs:
    # default hyperparameters for the Llama 7B model
    dim: int = 4096
    n_layers: int = 32
    n_heads: int = 32
    n_kv_heads: Optional[int] = None
    vocab_size: int = 32000
    hidden_dim: Optional[int] = None
    multiple_of: int = 256  # MLP hidden layer size will be multiple of
    norm_eps: float = 1e-5
    max_seq_len: int = 2048
    dropout: float = 0.0

In [3]:
args = ModelArgs()

Llama2的RMSNorm层的公式如下：

$$\text{RMSNorm}(x) = \frac{x}{\sqrt{\frac{1}{n}\sum_{i=1}^{n}w_i^2 + \epsilon}}$$

其中：

- ( $x$ ) 是层的输入。
- ( $w_i$ ) 代表层的权重。
- ( $n$ ) 是权重的数量。
- ( $\epsilon$ ) 是一个小常数，用于数值稳定性（以避免除以零的情况）。

In [4]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        output = self._norm(x.float()).type_as(x)
        return output * self.weight

In [5]:
norm = RMSNorm(dim=args.dim, eps=args.norm_eps)

In [6]:
# 写一个关于norm的测试

x = torch.randn(1, 50, 4096) # bs, seq_len, dim
print(x)
print(norm(x))

tensor([[[-1.0775,  0.5726, -0.6247,  ..., -0.5622, -0.2373,  1.5117],
         [-0.8565,  0.0924,  0.6640,  ...,  1.5508, -0.3674, -0.0278],
         [-0.0271,  0.6549,  1.0562,  ...,  0.3711,  1.8829,  2.1884],
         ...,
         [ 0.7752, -1.1106, -0.6566,  ..., -1.0418,  0.2752, -0.6765],
         [-0.8082,  1.6563, -0.0736,  ...,  0.9074,  0.7618,  0.5002],
         [-0.2666, -1.7303,  2.4673,  ..., -0.3045,  0.8025, -0.6515]]])
tensor([[[-1.0494,  0.5577, -0.6083,  ..., -0.5475, -0.2311,  1.4722],
         [-0.8461,  0.0913,  0.6559,  ...,  1.5320, -0.3630, -0.0275],
         [-0.0266,  0.6421,  1.0355,  ...,  0.3638,  1.8460,  2.1455],
         ...,
         [ 0.7662, -1.0978, -0.6490,  ..., -1.0298,  0.2721, -0.6687],
         [-0.8157,  1.6717, -0.0743,  ...,  0.9158,  0.7688,  0.5048],
         [-0.2678, -1.7377,  2.4779,  ..., -0.3058,  0.8060, -0.6543]]],
       grad_fn=<MulBackward0>)


假设 $\text{dim}$ 是输入维度，$\text{end}$ 是序列的长度，$\theta$ 是比例因子（默认为 10000.0）。

1. **频率计算**:
   $$\text{freqs} = \frac{1}{\theta^{\frac{2i}{\text{dim}}}}$$
   其中 $i = 0, 1, 2, ..., \frac{\text{dim}}{2} - 1$。

2. **时间序列与频率的外积**:
   创建一个从 0 到 $\text{end} - 1$ 的时间序列$t$，并计算 $t$ 和 $\text{freqs}$ 的外积得到频率矩阵。

3. **余弦和正弦值计算**:
   - 余弦值：$\text{freqs\_cos} = \cos(\text{freqs\_matrix})$
   - 正弦值：$\text{freqs\_sin} = \sin(\text{freqs\_matrix})$

其中，$\text{freqs\_matrix}$ 是时间序列 $t$ 和频率$ \text{freqs}$ 的外积的结果。


这个例子首先定义了函数 `precompute_freqs_cis`。然后，它设置了维度 `dim` 为 10，序列长度 `end` 为 5，并保持默认的比例因子$\theta = 10000.0$。通过调用这个函数并传入这些参数，它计算了序列中每个位置的余弦和正弦值。最后，这个例子打印了这些计算得到的余弦和正弦值矩阵。

这种预计算的余弦和正弦值可以用于例如 Transformer 模型中的位置编码，以提供位置信息，帮助模型理解输入数据中元素的顺序关系。

In [8]:
def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0):
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    t = torch.arange(end, device=freqs.device)  # type: ignore
    freqs = torch.outer(t, freqs).float()  # type: ignore
    freqs_cos = torch.cos(freqs)  # real part
    freqs_sin = torch.sin(freqs)  # imaginary part
    return freqs_cos, freqs_sin

In [12]:
theta=10000.0
dim=4096
end=50

freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
t = torch.arange(end, device=freqs.device)
res = torch.outer(t, freqs).float() 
freqs.shape, t.shape, res.shape

(torch.Size([2048]), torch.Size([50]), torch.Size([50, 2048]))

In [15]:
cos, sin = precompute_freqs_cis(4096, 50)
cos.shape, sin.shape    

(torch.Size([50, 2048]), torch.Size([50, 2048]))